<a href="https://colab.research.google.com/github/AmbiTyga/Bio-VI-BERT/blob/main/25GB_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>